<a href="https://colab.research.google.com/github/mprocz/Sistema_Imunologico_CVRP/blob/main/Sistema_Imunol%C3%B3gico_CVRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CVRPLIB: http://vrp.galgos.inf.puc-rio.br/index.php/en/

Biblioteca para ler os dados: https://github.com/PyVRP/VRPLIB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install vrplib
import random
import math
import vrplib
from random import shuffle, randint
from typing import List
import numpy as np
import numpy.typing as npt
import pandas as pd
import os

In [ ]:
class Data:
    name: str = ""
    trucks: int = 0
    dimension: int = 0
    truck_capacity: int = 0
    node_coord_section: np.array = []
    edge_weight_section: np.array = []
    demand: np.array = []

    def set_data(path):
        instance = vrplib.read_instance(path)
        Data.name = instance["name"]
        Data.trucks = int(instance['name'].split('-')[-1][1:])
        Data.dimension = instance["dimension"]
        Data.truck_capacity = instance["capacity"]
        Data.node_coord_section = np.array(instance["node_coord"])
        Data.edge_weight_section = np.array(instance["edge_weight"])
        Data.demand = np.array(instance["demand"])


In [ ]:
class Antibody:
    def __init__(self, route: List[int] = None, gen: int = 0):
        self.affinity: int = 0
        self.route: List[int] = route
        self.generate_route(route)
        self.generation = gen

    def __str__(self):
        return f"Antibody Affinity: {self.affinity}, generation: {self.generation}, antibody route: {self.route}"

    def generate_route(self, route: List[int]) -> None:
        if route is None:
            new_route = []
            for i in range(1, Data.dimension):
                new_route.append(i)
            for i in range(Data.trucks-1):
                new_route.append(0)
            shuffle(new_route)
            self.route = new_route
            self.affinity = self.calculate_affinity()
            self.check_feasible(new_route)
        else:
            self.affinity = self.calculate_affinity()
            self.check_feasible(route)

    def check_feasible(self, route: List[int]) -> None:
        current_load = 0
        penality = 0
        for node in route:
            if node > 0:
                current_load += Data.demand[node]
            else:
                current_load = 0
            if current_load > Data.truck_capacity:
                penality += current_load
        self.affinity += penality

    def calculate_affinity(self) -> int:
        new_affinity = 0
        new_affinity += (round(Data.edge_weight_section[0][self.route[0]], 0))
        new_affinity += (round(Data.edge_weight_section[self.route[-1]][0], 0))
        for i in range(len(self.route) - 1):
            new_affinity += (round(Data.edge_weight_section[self.route[i]][self.route[i + 1]], 0))
        return int(new_affinity)

    def hypermutation(self, number_of_exchanges: int, gen: int) -> None:
        for _ in range(number_of_exchanges):
            op = randint(0, 1)
            if op == 0:
                i = random.randint(0, len(self.route)-1)
                j = random.randint(0, len(self.route)-1)
                if self.route[i] == 0 or self.route[j] == 0: continue
                self.route[i], self.route[j] = self.route[j], self.route[i]
            elif op == 1:
                i, j = sorted(random.sample(range(len(self.route)), 2))
                if self.route[i] == 0 or self.route[j] == 0: continue
                self.route[i:j+1] = reversed(self.route[i:j+1])
        self.affinity = self.calculate_affinity()
        self.check_feasible(self.route)
        self.generation = gen

In [ ]:
class AIS:
    def __init__(self, population_size, generations, n_selected, n_replaced, cut_point):
        self.population_size = population_size
        self.generations = generations
        self.population = []
        self.n_selected: int = n_selected
        self.n_replaced: int = n_replaced
        self.cut_point: int = cut_point
        self.number_of_clones: int = 0
        self.bestaff = 10000

    def generate_population(self, gen: int = 0) -> None:
        difference = self.population_size - len(self.population)
        for _ in range(difference):
            self.population.append(Antibody(gen=gen))

    def clonal_algorithm(self) -> Antibody:
        self.generate_population()
        self.population.sort(key=lambda antibody: antibody.affinity)
        for generation in range(self.generations):
            print(f"Generation: {generation}, Best Affinity: {self.population[0].affinity}")

            clones = []

            if self.population[0].affinity < self.bestaff:
                self.bestaff = self.population[0].affinity
            elif self.population[0].affinity == self.bestaff:
                for _ in range(20):
                  clone = Antibody(route=self.population[0].route.copy())
                  clones.append(clone)
                  self.number_of_clones += 1

            for index, clone in enumerate(clones):
                clone.hypermutation(6, gen=generation)

            for i in range(self.n_selected):
                n_clones = self.population_size - i
                for _ in range(n_clones):
                  clone = Antibody(route=self.population[i].route.copy())
                  clones.append(clone)
                  self.number_of_clones += 1

            clones = clones + self.population[120:140]

            clones.sort(key = lambda x : x.affinity)
            for index, clone in enumerate(clones):
                if index < len(clones) * 0.33:
                    clone.hypermutation(1, gen=generation)
                elif index < len(clones) * 0.67:
                    clone.hypermutation(2, gen=generation)
                else:
                    clone.hypermutation(3, gen=generation)

            clones.sort(key=lambda antibody: antibody.affinity)
            self.population = sorted(self.population + clones[:self.n_replaced], key=lambda antibody: antibody.affinity)[:self.population_size]
            self.population = self.population[:self.cut_point]
            self.generate_population(gen=generation)
            self.population.sort(key=lambda antibody: antibody.affinity)

        return self.population[0]

In [ ]:
def print_trucks(route):
    truck_routes = []
    current_truck = []
    for node in route:
        if node == 0:
            if current_truck:
                truck_routes.append(current_truck)
            current_truck = []
        else:
            current_truck.append(node)
    if current_truck:
        truck_routes.append(current_truck)
    for i, truck_route in enumerate(truck_routes):
        print(f"Caminhão {i+1}: {truck_route}")

def calculate_truck_discharge(route):
    truck_discharges = {}
    current_truck = 1
    current_load = 0
    for node in route:
        if node == 0:
            if current_truck not in truck_discharges:
                truck_discharges[current_truck] = 0
            current_truck += 1
            current_load = 0
        else:
            current_load += Data.demand[node]
            truck_discharges[current_truck - 1] = current_load
    return truck_discharges

Data.set_data("/content/drive/MyDrive/instancesCVRP/A/instances/A-n32-k5.vrp")

ais = AIS(population_size=200, generations=500, n_selected=20, n_replaced=20, cut_point=50)
best_antibody = ais.clonal_algorithm()

truck_loads = calculate_truck_discharge(best_antibody.route)
print("\nCarga dos caminhões:")
for truck, load in truck_loads.items():
    print(f"Caminhão {truck +1}: {load}")
print("\nMelhor rota:")
print_trucks(best_antibody.route)
print("\nAfinidade da melhor rota:", best_antibody.affinity)

print('\n')
for i in (ais.population): print(i)

Generation: 0, Best Affinity: 2305
Generation: 1, Best Affinity: 1990
Generation: 2, Best Affinity: 1696
Generation: 3, Best Affinity: 1572
Generation: 4, Best Affinity: 1453
Generation: 5, Best Affinity: 1404
Generation: 6, Best Affinity: 1322
Generation: 7, Best Affinity: 1245
Generation: 8, Best Affinity: 1206
Generation: 9, Best Affinity: 1153
Generation: 10, Best Affinity: 1135
Generation: 11, Best Affinity: 1088
Generation: 12, Best Affinity: 1059
Generation: 13, Best Affinity: 1027
Generation: 14, Best Affinity: 1011
Generation: 15, Best Affinity: 992
Generation: 16, Best Affinity: 976
Generation: 17, Best Affinity: 964
Generation: 18, Best Affinity: 948
Generation: 19, Best Affinity: 934
Generation: 20, Best Affinity: 918
Generation: 21, Best Affinity: 898
Generation: 22, Best Affinity: 882
Generation: 23, Best Affinity: 867
Generation: 24, Best Affinity: 834
Generation: 25, Best Affinity: 822
Generation: 26, Best Affinity: 822
Generation: 27, Best Affinity: 813
Generation: 28,